In [63]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/alba/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [91]:
import requests
from pymongo import MongoClient
import pandas as pd

In [62]:
client = MongoClient()
db = client.get_database("api_chat")

# Try the endpoints

In [64]:
url="http://localhost:5000"

In [65]:
res=requests.get(url)

In [66]:
res.json()

{'welcome': 'Welcome to my API'}

### User endpoint

#### Add a user

In [67]:
lst=["Roberto", "Carla", "Ruben"]
for i in lst:
    params={"name":i}
    res=requests.get(url+f"/user/create/{i}", params=params)
    
res

<Response [200]>

In [68]:
res.json()

'The user 5fc5957cb4bb558622f95773 was added to users'

### Chat endpoints

#### Add a chat with users

In [95]:
params={"chat name": "songs", "list of users": ["5fc25fbf83625cc50f29b42b","5fc3f4b24c80ee9420ab05b1", "5fc57aaa134221b97058b52f", "5fc5870af1669c929407715f", "5fc5870af1669c9294077160", "5fc58710f1669c9294077164"]}
res=requests.get(url+f"/chat/create/songs", params=params)

res

<Response [200]>

In [96]:
res.json()

'The chat 5fc59c9710a1bdbbda17d338 was added to chats'

In [71]:
# In the browser
# localhost:5000/chat/create/<chatname>?user_id=5fc5870af1669c929407715f&user_id=5fc5870af1669c9294077160&user_id=5fc58710f1669c9294077164

#### Add a new user to the chat

In [31]:
res=requests.get(url+f"/chat/adduser/5fc57b46b6edb7d53a240cc9")
    
res

<Response [200]>

### Messages endpoints

#### Insert new message to chat

In [35]:
params={"text": "good night"}
res=requests.get(url+f"/messages/create/5fc57b46b6edb7d53a240cc9/5fc5870af1669c929407715f/good night", params=params)
    
res

<Response [200]>

In [36]:
res.json()

{'_id': {'$oid': '5fc58c7ef1669c929407716d'}}

#### Insert a new message if the user is in the chat

In [ ]:
# I stay out too late
# Got nothing in my brain
# Can't stop, won't stop moving
# It is like I got this music in my mind
# Saying it is gonna be alright
# I never miss a beat
# I'm lightning on my feet

In [107]:
params={"text": "I never miss a beat"}
res=requests.get(url+f"/messages/create/ifuser/5fc59d3d10a1bdbbda17d339/5fc58710f1669c9294077164/I never miss a beat", params=params)
    
res

<Response [200]>

In [108]:
res.json()

"The messaje {'text': 'I never miss a beat'} was added to chat 5fc59e6d10a1bdbbda17d33f"

In [111]:
params={"text": "hello"}
res=requests.get(url+f"/messages/create/ifuser/5fc57b46b6edb7d53a240cc9/5fc25f70aff2a631cc34f46b/hello", params=params)
    
res

<Response [200]>

In [112]:
res.json()

'This user (5fc25f70aff2a631cc34f46b) is not in this chat (5fc57b46b6edb7d53a240cc9)'

#### Get all messages from a chat

In [113]:
res=requests.get(url+f"/chat/list/5fc59d3d10a1bdbbda17d339")
    
res

<Response [200]>

In [114]:
res.json()

[{'_id': {'$oid': '5fc59da010a1bdbbda17d33a'},
  'message': {'text': 'I stay out too late'}},
 {'_id': {'$oid': '5fc59df310a1bdbbda17d33b'},
  'message': {'text': 'Got nothing in my brain'}},
 {'_id': {'$oid': '5fc59e0910a1bdbbda17d33c'},
  'message': {'text': "Can't stop, won't stop moving"}},
 {'_id': {'$oid': '5fc59e1f10a1bdbbda17d33d'},
  'message': {'text': "It's like I got this music in my mind"}},
 {'_id': {'$oid': '5fc59e5710a1bdbbda17d33e'},
  'message': {'text': 'Saying it is gonna be alright'}},
 {'_id': {'$oid': '5fc59e6d10a1bdbbda17d33f'},
  'message': {'text': 'I never miss a beat'}}]

### Sentiment Intensity Analyzer

In [115]:
sia = SentimentIntensityAnalyzer()

In [116]:
# Obtain the messages from one chat
text=res.json()

In [117]:
mssges=[]
for d in text:
    content=d["message"]['text']
    mssges.append(content)
    
mssges

['I stay out too late',
 'Got nothing in my brain',
 "Can't stop, won't stop moving",
 "It's like I got this music in my mind",
 'Saying it is gonna be alright',
 'I never miss a beat']

In [118]:
total_sc=[]
for i in mssges:
    score=sia.polarity_scores(i)
    total_sc.append(score)
total_sc

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 0.443, 'pos': 0.557, 'compound': 0.4168},
 {'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.3612},
 {'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'compound': 0.25},
 {'neg': 0.0, 'neu': 0.581, 'pos': 0.419, 'compound': 0.1139}]

In [119]:
# Better using pandas to calculate the average of the scores
total_score=pd.DataFrame(total_sc)

In [120]:
total_score

,neg,neu,pos,compound
0,0.0,1.000,0.000,0.0000
1,0.0,1.000,0.000,0.0000
2,0.0,0.443,0.557,0.4168
3,0.0,0.737,0.263,0.3612
4,0.0,0.714,0.286,0.2500
5,0.0,0.581,0.419,0.1139


In [121]:
total_score.mean()

neg         0.000000
neu         0.745833
pos         0.254167
compound    0.190317
dtype: float64